# Final Sheet: Neural Network Competition

In this worksheet 5 you will learn how to use and apply neural networks. Additionally you will participate in a small competition in order to find best synapse vector for the prediction of transmembrane domains given a set of proteins. For information please see the powerpoint presentation. The function to train the neural network from this worksheet was implement similarly to http://iamtrask.github.io/2015/07/12/basic-python-network/.

__!!!! Important please read through all the code provided !!!!__

##### Provided Code for Neural Network

The code below is the main part of this exercise. It contains the algorithm to train the neural network. Please read through the algorithm using the information provided in the slides. 

In [1]:
import numpy as np
import urllib.parse, urllib.request # python2 needs urllib2, which is now inside urllib
import lxml.etree as etree
import numpy as np

def nonlin(x,deriv=False):
    """
    Sigmoid function, defines the activation of a neuron given a specific weight of the synapse
    """
    if(deriv==True):
        return x*(1-x)
    return 1/(1+np.exp(-x))

def train_neural_network(input_matrix, output_vector, epsilon, rep, print_error=False):
    """
    This function aims to train a synapse using the training dataset composed of input_matrix + output_matrix 
    :param input_matrix: numpy array containing observed input 
    :param output_vector: numpy array containing reference output 
    :param rep: number of iterations to train the synapse
    :param print_error: optional argument to print the mean absolute error
    :return: numpy array corresponding of the trained synapse
    """
    np.random.seed(1)
    syn0 = 2*np.random.random((len(input_matrix[0]),1)) - 1
    for iter in range(rep):
        l0 = input_matrix
        l1 = nonlin(np.dot(l0,syn0))
        l1_error = output_vector-l1
        l1_delta = l1_error * nonlin(l1,True)
        syn0 += epsilon*np.dot(l0.T,l1_delta)
        
        
        if print_error and not (iter%5):
            print ("Mean absolute error:  " + str(np.mean(np.abs(l1_error))))
        
    return syn0

def run_neural_network(input_matrix, trained_synapse, trained_hidden=None):
    """
    Returns rather Y instead of what is written above.
    :param input_matrix: numpy array containing observed input 
    :param trained_synapse: numpy array corresponding of the trained synapse 
    :return: numpy array corresponding to predicted output
    OBS: trained_hidden is used only if you have a hidden layer (extra task at end of notebook!)
    """
    if trained_hidden == None:
        return nonlin(np.dot(input_matrix,trained_synapse)) # predicts the output with the logistic regression and the trained
    #synapse
    else:
        l0 = input_matrix
        l1 = nonlin(np.dot(l0,trained_synapse))
        l2 = nonlin(np.dot(l1,trained_hidden))
        return l2
    
def sse_neural_network_prediction(output_predicted, output_reference):
    """
    This function computes the sum of squared error between predicted output and reference output. 
    :param output_predicted: numpy array corresponding of predicted output 
    :param output_reference: numpy array containing reference output 
    :return: sum of square error between predicted output and reference output 
    """
    output_error = output_reference - output_predicted
    sse = np.sum(output_error**2)
    return sse


##### Provided Code for protein information

Below are functions that use uniprot ids in order to retrieve certain types of information. 

In [2]:
# Set of AA properties
hydrophobic = set(["A","I","L","M","H","F","R","Y"])
charged = set(["D","E","K","R"])
polar = set(["N","M","C", "S","Q","T","D","E","R","H","K"])
heavy = set(["Q","K","E","M","H","F","R","Y","W"])
hydrophathic = set(["A", "M","C", "F","L","V","I"])

# Get related xml from entry number
def get_uniprot_entry_xml(entry_id):
    
    url = 'http://www.uniprot.org/uniprot/' 
    params = {'format':'xml','query':entry_id}
    data = urllib.parse.urlencode(params).encode("utf-8")
    request = urllib.request.Request(url, data)
    response = urllib.request.urlopen(request)
    xml = etree.fromstring(response.read(200000))
    
    return xml.find('{http://uniprot.org/uniprot}entry')
       

# Function to get information related to entry xml 
def get_sequence_entry(entry_xml):
    return entry_xml.find("{http://uniprot.org/uniprot}sequence").text.replace("\n", "")

def get_membrane_entry(entry_xml, seq_len):
    membrane = "".zfill(seq_len)
    for feature_xml in entry_xml.findall('{http://uniprot.org/uniprot}feature'):
        if feature_xml.attrib["type"] == "transmembrane region":
            location_xml = feature_xml.find("{http://uniprot.org/uniprot}location")
            begin_xml = location_xml.find("{http://uniprot.org/uniprot}begin")
            end_xml = location_xml.find("{http://uniprot.org/uniprot}end")
            begin = int(begin_xml.attrib["position"])
            end = int(end_xml.attrib["position"])
            length = end - begin
            membrane = membrane[:begin] + ''.ljust(length,"1") + membrane[end:]
    return membrane
   
def convert_AA_to_binary_depending_property(seq, property_array):
    binary_seq = ''
    
    for aa in seq:
        if aa in property_array:
            binary_seq += "1"
        else:
            binary_seq += "0"
            
    return binary_seq

def get_protein_information(list_uniprot_id):
    '''
    Create a dictionary that maps each uniprot id with the related entry information. 
    Each entry information is stored into a dictionary containing the following 
    keys: sequence, membrane, hydrophobic, polar, heavy, charged, hydrophathic.
    Everything but the sequence is stored using the following format: for each position in the 
    sequence if the character has the attribute set to 1 else 0.  
    '''
    proteins = {}
    errorProt = []
    for uniprot_id in list_uniprot_id:
        entry = {}
        
        try:
            # get the xml result
            entry_xml =  get_uniprot_entry_xml(uniprot_id)

            # get info
            entry["sequence"] = get_sequence_entry(entry_xml)
            entry["membrane"] = get_membrane_entry(entry_xml, len(entry["sequence"]))
            entry["hydrophobic"] = convert_AA_to_binary_depending_property(entry["sequence"], hydrophobic)
            entry["polar"] = convert_AA_to_binary_depending_property(entry["sequence"], polar)
            entry["heavy"] = convert_AA_to_binary_depending_property(entry["sequence"], heavy)
            entry["charged"] = convert_AA_to_binary_depending_property(entry["sequence"], charged)
            entry["hydrophathic"] = convert_AA_to_binary_depending_property(entry["sequence"], hydrophathic)

            # Add the new entry to the training_dataset
            proteins[uniprot_id] = entry
        except:
            print("Error with: {}".format(uniprot_id))
            errorProt.append(uniprot_id)
        
    return proteins, errorProt

## Task 0: Understand the neural network

##### The code to create a neural network is provided, but a good comprehension of it is essential before going further. Task 0 should be well understood before starting working on the rest of the practical

In this section, we are using the following toy example to illustrate how the the neural network code is working.
<img src="toy.png" width="200px">
 




__A. Input dataset__

The input matrix has a (4,3) shape where each row represents a sample and each column represents an input.

In [3]:
input_matrix = np.array([[0,0,1],[1,1,1],[1,0,1],[0,1,1]])

__B. Output dataset__

The output matrix has a (4,1) shape where each row corresponds to a sample in the input matrix 
 and the column represents its related reference value.

In [4]:
output_matrix = np.array([[0,1,1,0]]).T

__C. Train the synapse using the training dataset (input_matrix + output_matrix )__


During this step, the synapse is modified in order to map each input values with its related reference
output. Once this is done, the synapse is called "trained" meaning that for any input the synapse
can predict (with errors) an ouptut value.


In [5]:
# We are training a0 and a1, so two parameters, which is the way we fit the sigmoid function
# Y so the prediction is known for the training set when we're training the beta
# The output matrix is the information on whether it is transmembrane a.a or not, Y = output membrane = 1 or 0 depending on
# whether it is transmembrane or not
# Inputs are probably X
repetition = 20  # Number of times to train the synapse
epsilon = 0.2  # Define the learning rate
train_synapse = train_neural_network(input_matrix, output_matrix, epsilon, repetition, print_error=True)

Mean absolute error:  0.5172082754380926
Mean absolute error:  0.4858763450355317
Mean absolute error:  0.45212768637768475
Mean absolute error:  0.42217780714553527


__D. Run the network using the trained synapse on the input dataset to predict__

During this step, we are using the trained synapse with some input in order to predict related ouptut values.


In [6]:
predicted_values = run_neural_network(input_matrix, train_synapse)
print(predicted_values)

[[0.3376799 ]
 [0.65923894]
 [0.543314  ]
 [0.45327736]]


__E. Test predicted result with the reference__

This function aims to look at how well you prediction are compared to the original reference values


In [7]:
error = sse_neural_network_prediction(predicted_values, output_matrix)
print (error)

0.6441682739105632


## Task 1: Training Dataset

The aim of this section is to create the training dataset with the required information for the neural network (see 'A. input dataset' and 'B. output dataset'). 
To proceed, we will first create a list of uniprot id to use as training proteins (think carefully which proteins to take). Then we will retreive the protein information (sequence, transmembrane region) from uniprot and convert it into a input/output readeable by the neural network.

#### Create a list of uniprot entry id that you will use to build your training dataset

In [8]:
all_human_transmembrane_protein = []
# Read in possible human ids of proteins with transmembrane domain
with open('uniprot.txt') as f:
    for l in f:
        all_human_transmembrane_protein.append(l.strip().split("\t")[0])

#-----your code: -----
'''
Fill list_entry_training with uniprot ids of your choice. You should think about what to include into your
training dataset..
=> Let's include the reviewed sequences only => only one sequence is not reviewed
'''
reviewed_human_proteins = []
with open('uniprot.txt') as f:
    for l in f:
        if "reviewed" in l:
            reviewed_human_proteins.append(l.strip().split("\t")[0])

#Let's take 1000 random elements
list_entry_training = [reviewed_human_proteins[i] for i in np.random.choice(len(reviewed_human_proteins), size=1000, replace=False)]


# --------

print("The following entries will be used to train your neural network", list_entry_training )


The following entries will be used to train your neural network ['Q05901', 'Q9Y2C5', 'A6NDL8', 'Q96RV3', 'Q8NGC9', 'P01375', 'A6NJW4', 'Q8NGX8', 'I3L273', 'P51809', 'Q969E2', 'Q9NP91', 'P13637', 'Q8NHC6', 'P22888', 'Q16570', 'P48995', 'Q8IZY2', 'Q15375', 'Q8WYK1', 'Q6UWB1', 'Q86SU0', 'Q8N816', 'Q13467', 'P17927', 'Q16655', 'Q9BZJ8', 'O15247', 'Q9H7F0', 'Q96ES6', 'P30498', 'P30542', 'P32927', 'A6NJY4', 'Q9P003', 'Q9BUM1', 'Q70HW3', 'Q8IZF6', 'P36537', 'O95473', 'O60427', 'Q7RTS6', 'P16671', 'Q8NH70', 'P09564', 'O14967', 'P40197', 'Q9H819', 'Q8WWQ8', 'Q8NGI1', 'Q15822', 'Q07699', 'Q15615', 'Q9UNE0', 'Q8NH40', 'P27544', 'Q9H7M9', 'Q6PXP3', 'Q9Y5Y5', 'O43613', 'A6NNB3', 'P15813', 'Q9NS62', 'Q99678', 'Q8N6Q1', 'Q96T53', 'Q8NBR0', 'O95528', 'A6NLE4', 'Q96IW7', 'P30954', 'Q6YBV0', 'Q15743', 'Q9C091', 'A6NMD0', 'Q9BYE9', 'Q13410', 'Q8WUS8', 'Q9HBL7', 'P0C7N1', 'P36894', 'Q9NX62', 'Q5T7P8', 'P0C7T3', 'Q9H1U4', 'Q96EP9', 'Q9BZJ6', 'Q9BV87', 'Q9NVA4', 'Q68CR7', 'Q8NGJ8', 'Q8N1N2', 'Q8TD20', 'Q168

#### Fetch all the protein information from uniprot for all the entries in the list_entry_training

In [9]:
training_dataset, errorProt = get_protein_information(list_entry_training)

'''
Example of result return by get_protein_information(["P9KL37"]):

training_dataset = {
'P9KL37': {'polar': '10101...','sequence': 'MAVMA...','membrane': '00111...',
'charged': '00010...','hydrophobic': '000001...','heavy': '000001...','hydrophathic': '000001...'}}
'''
if errorProt:
    for prot in errorProt: # remove the prots that are not in the training dataset
        list_entry_training.remove(prot)

el_1 = training_dataset[list_entry_training[0]]
for k,v in el_1.items():
    print (k, "::\n", v) # prints just first element
# print (training_dataset) # prints whole dictionary

Error with: P21802
Error with: P00533
sequence ::
 MLPDFMLVLIVLGIPSSATTGFNSIAENEDALLRHLFQGYQKWVRPVLHSNDTIKVYFGLKISQLVDVDEKNQLMTTNVWLKQEWTDHKLRWNPDDYGGIHSIKVPSESLWLPDIVLFENADGRFEGSLMTKVIVKSNGTVVWTPPASYKSSCTMDVTFFPFDRQNCSMKFGSWTYDGTMVDLILINENVDRKDFFDNGEWEILNAKGMKGNRRDGVYSYPFITYSFVLRRLPLFYTLFLIIPCLGLSFLTVLVFYLPSDEGEKLSLSTSVLVSLTVFLLVIEEIIPSSSKVIPLIGEYLLFIMIFVTLSIIVTVFVINVHHRSSSTYHPMAPWVKRLFLQKLPKLLCMKDHVDRYSSPEKEESQPVVKGKVLEKKKQKQLSDGEKVLVAFLEKAADSIRYISRHVKKEHFISQVVQDWKFVAQVLDRIFLWLFLIVSVTGSVLIFTPALKMWLHSYH
membrane ::
 00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000111111111111111111111111000000001111111111111111100000000000000000111111111111111111111000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000011111111111111111100000000000
hydrophobic ::
 11

#### Create the input matrix using the training_dataset


In [43]:
observed_properties = ['polar','heavy']
sliding_window_size = 3

def getAllSlidingWindows(booleanSeq, sliding_window_size): # Get all the sliding windows for a sample
    listSlidingWindows = [booleanSeq[i:i+sliding_window_size] for i in range(len(booleanSeq)-sliding_window_size)]
    return listSlidingWindows

def sumListsOfStrings(listOfListsOfStrings): # Used to sum each feature string
    summedList = [""]*len(listOfListsOfStrings[0])
    for i in range(len(listOfListsOfStrings[0])):
        for j in range(len(listOfListsOfStrings)):
            summedList[i] += listOfListsOfStrings[j][i]
    return summedList

def get_input_matrix(entry_dict, sliding_window_size, properties):
    '''
     return a matrix with following shape: 
     -rows = samples, i.e. sliding window for all sequences in the training dataset
     -colums = combination of property and position within sliding window
    '''
    #-----your code: -----
    # Summing the sequences of the different samples per feature -> expected output is a list of sequences for diff features
    listOfAllSequences = []
    for feature in properties:
        fullSequence = "" # to all the sequences of samples
        for keys in entry_dict.keys():
            fullSequence += entry_dict[keys][feature]
        listOfAllSequences.append(fullSequence)
    
    # Turn the sequences into lists of all their sliding windows
    listsOfSlidingWindows = []
    for seqPerFeature in listOfAllSequences:
        listsOfSlidingWindows.append(getAllSlidingWindows(seqPerFeature, sliding_window_size))
    
    # Build the input vector containing, which is a vector of the positions of all sliding windows combined for all features
    # Example of expected output (vertical): v = ["110010", "101000", ...] with each values being the combination of the
    #sliding windows of 2 features for all the sequences
    input_vector = sumListsOfStrings(listsOfSlidingWindows)
    
    # Build the input_matrix which decompose each row of the vector into a list of integer of its components
    # ex: "110010" -> [1,1,0,0,1,0]
    input_matrix = [[int(i) for i in binarySequence] for binarySequence in input_vector]
    # --------
    return np.array(input_matrix)



input_matrix = get_input_matrix(training_dataset,sliding_window_size,observed_properties)
print(input_matrix.shape)
print(input_matrix[0:10])
# In case I would have misunderstood what is asked: The matrix I generate is a matrix with columns: all possible sliding
# windows of the combined features: ex: first sliding windows of 3 positions with 2 features will be sliding window1 + sliding window2
# whereas the rows are samples

(546589, 6)
[[1 0 0 1 0 0]
 [0 0 1 0 0 0]
 [0 1 0 0 0 1]
 [1 0 1 0 1 1]
 [0 1 0 1 1 0]
 [1 0 0 1 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


##### Create the output matrix using the  training_dataset

In [75]:
value_to_predict = 'membrane'


def get_output_matrix(entry_dict,sliding_window_size, value_to_predict):
    '''
     return a vector that contains for each sliding windows the related observed value (for the value_to_predict)
     
    '''
    out_vector = ""
    #-----your code: -----
    # Summing the sequences of the different samples for membrane, expected output is a long sequence being the fusion of all
    # the sample membrane sequences
    fullSequence = "" # to add all the sequences of samples
    for keys in entry_dict.keys():
        fullSequence += entry_dict[keys][value_to_predict]

    # Turn the sequence into a list of all its sliding windows
    allSlidingWindows = getAllSlidingWindows(fullSequence, sliding_window_size)
    
    # Turn the list of sliding windows into a list of the middle value of each sliding window -> sliding windows must be impair
    out_vector_list = [int(elem[len(elem)//2]) for elem in allSlidingWindows]
    # --------
    return np.array([out_vector_list]).T
    

output_matrix = get_output_matrix(training_dataset,sliding_window_size, value_to_predict)
print (output_matrix.shape)
print(output_matrix[0:10])

# To summarize: each row of the input is a sliding window being the combination of 2 sliding windows of the same positions in
# sequences for different features. Each row of the output is the corresponding output, being the middle value of the 
# sliding windows of the feature of interest

(546583, 1)
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]


## Task 2: Train the Neural Network

The aim of this section is to train a synapse based on the training dataset (see 'C. Train the synapse')

In [112]:
train_synapse = None

#-----change parameters to find best syn0: -----
repetition = 10000
observed_properties = ['polar','heavy','hydrophobic']
sliding_window_size = 9
epsilon = 0.005
# input data variation has to be defined at the beginning of these sheet


#-----your code: -----
'''
Create the input_matrix and output_matrix for the training dataset then train your synapse on it.
     
'''
# building input matrix
input_matrix = get_input_matrix(training_dataset, sliding_window_size, observed_properties)
print(input_matrix.shape)
# print(len(input_matrix[0])) # Long sliding window as expected

# building output matrix
output_vector = get_output_matrix(training_dataset, sliding_window_size, 'membrane')
print(output_matrix.shape)

# Train the synapse based on the training dataset
syn0 = train_neural_network(input_matrix, output_vector, epsilon, repetition, print_error=False)

# --------


(546583, 27)


/home/boris/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in exp
  if sys.path[0] == '':


(546583, 1)


## Task 3: Benchmarck Predicted Values

The aim of this section is to assess the quality the prediction we are doing based on our trained synapse are similar to the reference values (see 'D. Run the network' and 'E. Test predicted result')

##### Provided code



In [113]:
def round_int(in_l1,cutoff):
    out_l1 = [0]*len(in_l1)
    for i in range(0,len(in_l1)-1):
        tmp = round(in_l1[i],cutoff)
        if cutoff == 0:
            out_l1[i] = int(tmp)
        else:
            out_l1[i] = tmp
    return out_l1

def pretty_print_output_matrix(output_matrix):
    return ''.join(str(x) for x in round_int(output_matrix,0))
    

def calculate_fp_fn_tp_tn(output, output_predicted): # To calculate the number of true positive, false negative, ...
    result = {"fn":0, "fp":0, "tp":0, "tn":0}
    for i in range(0,len(output)):
        observed = int(round(int(output_predicted[i]),0))
        reference = int(round(float(output[i]),0))
        if reference==1 and observed==1:
            result["tp"] += 1
        if reference==1 and observed==0:
            result["fn"] += 1
        if reference==0 and observed==1:
            result["fp"] += 1 
        if reference==0 and observed==0:
            result["tn"] +=1
    return result

### Task 3.1: Benchmarck using set of given proteins to determine accuracy of prediction

In the following section, we will use our trained synapse on a bunch of proteins in order to predict transmembrane region. For each prediction, we will then compare it to the curated transmembrane region annotation obtained from uniprot. To assess we will use 3 scoring methods: the precision, the recall, and the sse.

In [114]:
print(syn0) # The values are all negative and range from ~-7 to ~-52

[[-43.18371134]
 [-51.54129928]
 [-25.76439503]
 [-28.94009384]
 [-39.83455422]
 [-23.62648792]
 [-28.99741232]
 [-31.06545538]
 [-43.04155734]
 [-24.58737413]
 [-24.80347511]
 [-27.25960474]
 [-19.75104848]
 [-32.96488107]
 [-11.38919797]
 [-28.75972411]
 [-23.87089217]
 [-23.54704106]
 [-10.33964046]
 [ -7.28000558]
 [-34.68193244]
 [-33.17226734]
 [-18.42392626]
 [-26.79368249]
 [-35.46521494]
 [-31.8315825 ]
 [ -8.88109056]]


In [115]:
test_id = ["P13569","Q4KMQ2","P03989","Q9H221","P28335"]
test_dataset = get_protein_information(test_id)

# Test that the input is really a dictionary and otherwise remove the value that is not in the dictionary
print(isinstance(test_dataset, dict)) # If return false, the file needs to be changed -> we see that the error occurs with
# P13569 so let's remove it from the list and take the dictionary without it

test_id = [elem for elem in test_id if elem != "P13569"]

test_dataset = test_dataset[0]
print(test_dataset) # This time seems to be a dictionary

Error with: P13569
False
{'Q4KMQ2': {'sequence': 'MKKMSRNVLLQMEEEEDDDDGDIVLENLGQTIVPDLGSLESQHDFRTPEFEEFNGKPDSLFFNDGQRRIDFVLVYEDESRKETNKKGTNEKQRRKRQAYESNLICHGLQLEATRSVLDDKLVFVKVHAPWEVLCTYAEIMHIKLPLKPNDLKNRSSAFGTLNWFTKVLSVDESIIKPEQEFFTAPFEKNRMNDFYIVDRDAFFNPATRSRIVYFILSRVKYQVINNVSKFGINRLVNSGIYKAAFPLHDCKFRRQSEDPSCPNERYLLYREWAHPRSIYKKQPLDLIRKYYGEKIGIYFAWLGYYTQMLLLAAVVGVACFLYGYLNQDNCTWSKEVCHPDIGGKIIMCPQCDRLCPFWKLNITCESSKKLCIFDSFGTLVFAVFMGVWVTLFLEFWKRRQAELEYEWDTVELQQEEQARPEYEARCTHVVINEITQEEERIPFTAWGKCIRITLCASAVFFWILLIIASVIGIIVYRLSVFIVFSAKLPKNINGTDPIQKYLTPQTATSITASIISFIIIMILNTIYEKVAIMITNFELPRTQTDYENSLTMKMFLFQFVNYYSSCFYIAFFKGKFVGYPGDPVYWLGKYRNEECDPGGCLLELTTQLTIIMGGKAIWNNIQEVLLPWIMNLIGRFHRVSGSEKITPRWEQDYHLQPMGKLGLFYEYLEMIIQFGFVTLFVASFPLAPLLALVNNILEIRVDAWKLTTQFRRLVPEKAQDIGAWQPIMQGIAILAVVTNAMIIAFTSDMIPRLVYYWSFSVPPYGDHTSYTMEGYINNTLSIFKVADFKNKSKGNPYSDLGNHTTCRYRDFRYPPGHPQEYKHNIYYWHVIAAKLAFIIVMEHVIYSVKFFISYAIPDVSKRTKSKIQREKYLTQKLLHENHLKDMTKNMGVIAERMIEAVDNNLRPKSE', 'membrane': '000000000000000000000000

In [116]:
precisions = []
recalls = []
sses = []

def get_input_matrix2(entry_dict, sliding_window_size, properties): # entry_dict must this time be one of the sub-dictionary
    '''
     return a matrix with following shape: 
     -rows = samples, i.e. sliding window for the sequence
     -colums = combination of property and position within sliding window
    '''
    #-----your code: -----
    # Build a list containing the different property sequences
    listOfAllSequences = [entry_dict[feature] for feature in properties]
    
    # Turn the sequences into lists of all their sliding windows
    listsOfSlidingWindows = []
    for seqPerFeature in listOfAllSequences:
        listsOfSlidingWindows.append(getAllSlidingWindows(seqPerFeature, sliding_window_size))
    
    # Build the input vector containing, which is a vector of the positions of all sliding windows combined for all features
    # Example of expected output (vertical): v = ["110010", "101000", ...] with each values being the combination of the
    #sliding windows of 2 features for all the sequences
    input_vector = sumListsOfStrings(listsOfSlidingWindows)
    
    # Build the input_matrix which decompose each row of the vector into a list of integer of its components
    # ex: "110010" -> [1,1,0,0,1,0]
    input_matrix = [[int(i) for i in binarySequence] for binarySequence in input_vector]
    # --------
    return np.array(input_matrix)

def get_output_matrix2(entry_dict,sliding_window_size, value_to_predict): # For individual sub-dictionaries
    '''
     return a vector that contains for each sliding windows the related observed value (for the value_to_predict)
     
    '''
    out_vector = ""
    #-----your code: -----
    # Get only one feature binary sequence from all the feature sequences
    fullSequence = entry_dict[value_to_predict]

    # Turn the sequence into a list of all its sliding windows
    allSlidingWindows = getAllSlidingWindows(fullSequence, sliding_window_size)
    
    # Turn the list of sliding windows into a list of the middle value of each sliding window -> sliding windows must be impair
    out_vector_list = [int(elem[len(elem)//2]) for elem in allSlidingWindows]
    # --------
    return np.array([out_vector_list]).T


for entry, information in test_dataset.items(): # Entry is the prot ID and info is the dictionary with the different
    #information such as, membrane, hydrophobic, etc
    print("Benchmarking of entry: ", entry)
    
    #-----your code: -----
    # Need at each step to generate the input matrix from a new function as it is directed against the sequence of only 1 sample
    # on each sample one after the other, so retake the matrix function but remove the part where we sum per sampple
    input_matrix2 = get_input_matrix2(test_dataset[entry], sliding_window_size, observed_properties)
    # Then run the neural network with the new input matrix
    output_predicted = run_neural_network(input_matrix2, syn0, trained_hidden=None)
    
    # Get the reference outputs for each of the protein, which is the 'membrane' binary sequence
    output_reference = get_output_matrix2(test_dataset[entry], sliding_window_size, value_to_predict) # last argument defined as 
    #'membrane' before
    print(output_predicted, "\n\n", output_reference)
    # Calculate sse
    sse = sse_neural_network_prediction(output_predicted, output_reference)

    # Calculate precision = True positive rate = true_positives/total_positives
    list_T_F_Rates = calculate_fp_fn_tp_tn(output_reference, output_predicted)
    #precision = list_T_F_Rates["tp"]/(list_T_F_Rates["tp"]+list_T_F_Rates["fp"])
    print(list_T_F_Rates)
    # Calculate recall = true_positives/total_True
    #recall = list_T_F_Rates["tp"]/(list_T_F_Rates["tp"]+list_T_F_Rates["fn"])
    
    
    # --------
    print("the sse is equal to ", sse)  
    #print("The precision is ", precision)
    #print("The recall is ", recall)
    print("\n")
    
    precisions.append(precision)
    #recalls.append(recall)
    #sses.append(sse)


#print("the average sse is equal to ", sum(sses)/len(sses) )
#print("The average precision is ", sum(precisions)/len(precisions))
#print("The average recall is ", sum(recalls)/len(recalls))

Benchmarking of entry:  Q4KMQ2
[[5.57380057e-187]
 [1.55814092e-181]
 [1.15301157e-188]
 [2.29507969e-191]
 [2.60123568e-178]
 [6.51948557e-188]
 [2.85606567e-185]
 [8.78469444e-173]
 [2.03538272e-181]
 [1.06060976e-201]
 [2.97914108e-202]
 [1.23473670e-198]
 [3.90121553e-161]
 [9.28573567e-158]
 [5.54721205e-132]
 [5.10279533e-120]
 [3.83163240e-095]
 [9.82876565e-125]
 [2.17724568e-120]
 [3.81212066e-099]
 [8.46254109e-088]
 [3.01091982e-123]
 [6.50947426e-113]
 [6.42984587e-090]
 [1.35754352e-105]
 [6.94263421e-125]
 [2.03519201e-091]
 [3.65614708e-066]
 [6.76583994e-086]
 [5.82380028e-112]
 [6.29573139e-069]
 [4.00738443e-081]
 [1.54763604e-094]
 [1.32809266e-134]
 [1.19893671e-127]
 [3.77822557e-154]
 [6.91064078e-160]
 [2.92892996e-186]
 [7.43141268e-205]
 [5.52563343e-199]
 [5.71337451e-194]
 [1.80055067e-188]
 [5.56256439e-215]
 [5.96869183e-180]
 [1.25820392e-174]
 [1.13229080e-183]
 [1.06075051e-153]
 [1.21685607e-171]
 [1.05222013e-144]
 [3.86098349e-156]
 [3.55222605e-150]


__Provided code__

In [ ]:
from Bio import SeqIO
fasta_a = "small_A.fa"

# parse the fasta file and load it into the records object
handle = open(fasta_a, "r")
records = list(SeqIO.parse(handle, "fasta"))
handle.close()


def parse_single_seq(record):
    hydrophobic = ["A","I","L","M","H","F","R","Y"]
    charged = ["D","E","K","R"]
    polar = ["N","M","C", "S","Q","T","D","E","R","H","K"]
    heavy = ["Q","K","E","M","H","F","R","Y","W"]
    hydrophathic = ["A", "M","C", "F","L","V","I"]
    entries = {}
    entry = {}
    entry["sequence"] = str(record.seq)
    entry["hydrophobic"] = str(record.seq)
    entry["polar"] = str(record.seq)
    entry["charged"] = str(record.seq)
    entry["heavy"] = str(record.seq)
    entry["hydrophathic"] = str(record.seq)
    
    for aa_polar in polar:
        entry["polar"] = entry["polar"].replace(aa_polar, "1")
    for aa in entry["polar"]:
        if aa !='1':
            entry["polar"] = entry["polar"].replace(aa, "0")
            
    for aa_charged in hydrophobic:
        entry["hydrophobic"] = entry["hydrophobic"].replace(aa_charged, "1")
    for aa in entry["hydrophobic"]:
        if aa !='1':
            entry["hydrophobic"] = entry["hydrophobic"].replace(aa, "0")   
            
    for aa_charged in charged:
        entry["charged"] = entry["charged"].replace(aa_charged, "1")
    for aa in entry["charged"]:
        if aa !='1':
            entry["charged"] = entry["charged"].replace(aa, "0")
            
    for aa_heavy in heavy:
        entry["heavy"] = entry["heavy"].replace(aa_heavy, "1")
    for aa in entry["heavy"]:
        if aa !='1':
            entry["heavy"] = entry["heavy"].replace(aa, "0")
            
    for aa_hydrophathic in hydrophathic:
        entry["hydrophathic"] = entry["hydrophathic"].replace(aa_hydrophathic, "1")
    for aa in entry["hydrophathic"]:
        if aa !='1':
            entry["hydrophathic"] = entry["hydrophathic"].replace(aa, "0")
            
    entries[record.name] = entry
    
    return entries

# Function to get information related to entry xml 
def get_num_membrane_entry(entry_xml):
    num_membrane = []
    for feature_xml in entry_xml.findall('{http://uniprot.org/uniprot}feature'):
        if feature_xml.attrib["type"] == "transmembrane region":
            location_xml = feature_xml.find("{http://uniprot.org/uniprot}location")
            begin_xml = location_xml.find("{http://uniprot.org/uniprot}begin")
            begin = int(begin_xml.attrib["position"])
            num_membrane.append(begin)
    return len(num_membrane)

### Task 3.2: Benchmarck using fasta file to count number of correctly identified sequences that have domain

In the following section, we will use our trained synapse on a fasta file with random sequences and count the number of correctly identified sequences vs the number of all sequences that have a transmembrane domain

In [ ]:
total_mem_ids = 0.0
found_mem_ids = 0.0

for i in range(0,50):
    l0 = get_input_matrix(parse_single_seq(records[i]),sliding_window_size,obversed_properties)
    l1 = run_neural_network(test_input,train_synapse,train_hidden)
    test = pretty_print_output_matrix(l1)
    entry_xml =  get_uniprot_entry_xml(records[i].name.split("|")[1])
    num_membrane = get_num_membrane_entry(entry_xml)
    
    if num_membrane > 0:
        total_mem_ids += 1
        if '111111111' in test:
            found_mem_ids += 1

print("The percentage of correctly identified sequences with transmembrane domains is ", found_mem_ids/total_mem_ids) 


## Optional: Neural Network with Hidden Layers 

Obviously there are better ways of doing a neural network and there is plenty of research going on to improve the ability to learn. One simple way of doing is to add another level of abstraction which is a hidden layer. While not being part of this excersice we still provide with our implementation (quite similar to http://iamtrask.github.io/2015/07/12/basic-python-network/). 

In [ ]:
print('''If you have finish everything above, try to add an hidden layer in your neural network. 
      Checking the blog post should be a good start... ''')

def train_neural_network_hidden(input_matrix,output_vector,epsilon,rep):
    
    np.random.seed(1)
    syn0 = 2*np.random.random((len(input_matrix[0]),3)) - 1
    syn1 = 2*np.random.random((3,1)) - 1
    for iter in xrange(rep):
        l0 = input_matrix
        
        l1 = nonlin(np.dot(l0,syn0))
        l2 = nonlin(np.dot(l1,syn1))
        
        l2_error = output_vector - l2
        l2_delta = l2_error*nonlin(l2,deriv=True)
        syn1 += epsilon*np.dot(l1.T,l2_delta)
        
        l1_error = l2_delta.dot(syn1.T)
        l1_delta = l1_error*nonlin(l1,deriv=True)
        syn0 += epsilon*np.dot(l0.T,l1_delta)
        
        if (iter% 1000) == 0:
            print "Error:" + str(np.mean(np.abs(l2_error)))                

    return syn0,syn1


# END